In [ ]:
    @staticmethod
    async def process_async_stream_response(response: aiohttp.ClientResponse) -> Optional[str]:
        """
        Process the asynchronous streaming response from the CMS API.
        
        Args:
            response: The response object from async_post_stream
            
        Returns:
            The final answer from the streamed response
        """
        if response is None:
            logger.error("No response object provided for streaming")
            return None
            
        buffer = ""
        chunk_size = getattr(response, "_chunk_size", 4096)
        
        try:
            # Process the stream in chunks
            async for chunk in response.content.iter_chunked(chunk_size):
                if chunk:
                    data = chunk.decode("utf-8", errors="ignore")
                    buffer += data
                    
                    # Process complete JSON objects from the buffer
                    while buffer:
                        try:
                            json_data, index = json.JSONDecoder().raw_decode(buffer)
                            buffer = buffer[index:].lstrip()
                            
                            is_streaming = json_data.get("streaming", False)
                            if is_streaming:
                                if "stream_data" in json_data and "text" in json_data["stream_data"]:
                                    streamed_text = json_data["stream_data"]["text"]
                                    print(streamed_text, end="")
                            else:
                                # Final response found
                                if "final_response" in json_data and "answer" in json_data["final_response"]:
                                    final_response = json_data["final_response"]
                                    answer = final_response["answer"]
                                    return answer
                        except json.JSONDecodeError:
                            # Incomplete JSON, wait for more data
                            break
        except Exception as e:
            logger.error(f"Error processing stream: {str(e)}")
        finally:
            # Always close the response and session
            try:
                if hasattr(response, "_session"):
                    await response._session.close()
            except Exception as e:
                logger.error(f"Error closing stream session: {str(e)}")
        
        return None
    
    @staticmethod
    async def process_async_stream_response(response: aiohttp.ClientResponse) -> Optional[str]:
        """
        Process the asynchronous streaming response from the CMS API.
        
        Args:
            response: The response object from async_post_stream
            
        Returns:
            The final answer from the streamed response
        """
        if response is None:
            logger.error("No response object provided for streaming")
            return None
            
        buffer = ""
        chunk_size = getattr(response, "_chunk_size", 4096)
        
        try:
            # Process the stream in chunks
            async for chunk in response.content.iter_chunked(chunk_size):
                if chunk:
                    data = chunk.decode("utf-8", errors="ignore")
                    buffer += data
                    
                    # Process complete JSON objects from the buffer
                    while buffer:
                        try:
                            json_data, index = json.JSONDecoder().raw_decode(buffer)
                            buffer = buffer[index:].lstrip()
                            
                            is_streaming = json_data.get("streaming", False)
                            if is_streaming:
                                if "stream_data" in json_data and "text" in json_data["stream_data"]:
                                    streamed_text = json_data["stream_data"]["text"]
                                    print(streamed_text, end="")
                            else:
                                # Final response found
                                if "final_response" in json_data and "answer" in json_data["final_response"]:
                                    final_response = json_data["final_response"]
                                    answer = final_response["answer"]
                                    return answer
                        except json.JSONDecodeError:
                            # Incomplete JSON, wait for more data
                            break
        except Exception as e:
            logger.error(f"Error processing stream: {str(e)}")
        finally:
            # Always close the response and session
            try:
                if hasattr(response, "_session"):
                    await response._session.close()
            except Exception as e:
                logger.error(f"Error closing stream session: {str(e)}")
        
        return None